In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [13]:
# For reproducibility
torch.manual_seed(1)

$ P(class=i) = \frac{e^i}{\sum e^i} $ : 수치를 확률값으로 반환

In [4]:
z = torch.FloatTensor([1, 2, 3])

In [5]:
hypothesis = F.softmax(z, dim=0)  # pytorch에서 제공 
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [6]:
hypothesis.sum()

tensor(1.)

### Cross Entropy Loss (Low-level)
For multi-class classification, we use the cross entropy loss.

$ L = \frac{1}{N} \sum - y \log(\hat{y}) $

$\hat y $ : predicted probability, $y$ : correct probability(0 or 1)

In [42]:
z = torch.rand(3, 5, requires_grad = True)
hypothesis = F.softmax(z,dim = 1)
print(hypothesis)

tensor([[0.1703, 0.2118, 0.1857, 0.1963, 0.2359],
        [0.2328, 0.2268, 0.1060, 0.2356, 0.1987],
        [0.3189, 0.2760, 0.1258, 0.1234, 0.1559]], grad_fn=<SoftmaxBackward>)


In [43]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([3, 1, 2])


In [44]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [45]:
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # scatter_(dim(차원축),index(위치),src(scatter할 값))

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [52]:
# scatter_를 이용한 one hot encoding
label = torch.tensor([3, 4, 5, 6, 7])
one_hot = torch.zeros(5,10)
label = label.view(-1,1)
print(one_hot)
print(label) # 해당 인덱스에 1을 채움 
print(one_hot.scatter_(1, label, 1)) # row별로 1을 해당 인덱스에 삽입

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[3],
        [4],
        [5],
        [6],
        [7]])
tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])


In [21]:
# cross entropy loss 
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


### Cross-entropy Loss with torch.nn.functional
PyTorch의 F.log_softmax() 함수 제공

In [26]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [27]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [28]:
# Low level
(y_one_hot * -torch.log(F.softmax(z,dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [30]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [31]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

## Training with Low-level Cross Entropy Loss

In [53]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [33]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1) :
    
    # Cost 
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim = 1))).sum(dim=1).mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost{:.6f}'.format(epoch, nb_epochs, cost.item()))


Epoch    0/1000 Cost1.098612
Epoch  100/1000 Cost0.901535
Epoch  200/1000 Cost0.839114
Epoch  300/1000 Cost0.807826
Epoch  400/1000 Cost0.788472
Epoch  500/1000 Cost0.774822
Epoch  600/1000 Cost0.764449
Epoch  700/1000 Cost0.756191
Epoch  800/1000 Cost0.749398
Epoch  900/1000 Cost0.743671
Epoch 1000/1000 Cost0.738749


### Training with F.cross_entropy

In [46]:
# 모델 초기화
W = torch.zeros((4,3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr =0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1) :
    
    # Cost 계산
    z = x_train.matmul(W) + b 
    cost = F.cross_entropy(z, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력 
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost : 1.098612
Epoch  100/1000 Cost : 0.761050
Epoch  200/1000 Cost : 0.689991
Epoch  300/1000 Cost : 0.643229
Epoch  400/1000 Cost : 0.604117
Epoch  500/1000 Cost : 0.568255
Epoch  600/1000 Cost : 0.533922
Epoch  700/1000 Cost : 0.500291
Epoch  800/1000 Cost : 0.466908
Epoch  900/1000 Cost : 0.433507
Epoch 1000/1000 Cost : 0.399963


### High-level Implementation with nn.Module

In [54]:
class SoftmaxClassifierModel(nn.Module) :
    def __init__(self) : 
        super().__init__()
        self.linear = nn.Linear(4,3) 
    
    def forward(self, x) :
        return self.linear(x)

In [55]:
model = SoftmaxClassifierModel()

In [56]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1) :
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.cross_entropy(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost : {:.6f}'.format( epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost : 1.193624
Epoch  100/1000 Cost : 0.647261
Epoch  200/1000 Cost : 0.559104
Epoch  300/1000 Cost : 0.503264
Epoch  400/1000 Cost : 0.458217
Epoch  500/1000 Cost : 0.418269
Epoch  600/1000 Cost : 0.380892
Epoch  700/1000 Cost : 0.344497
Epoch  800/1000 Cost : 0.307936
Epoch  900/1000 Cost : 0.271172
Epoch 1000/1000 Cost : 0.243551
